## Description:
"
    "Interpolates Nek output files onto a structured grid and writes the results to HDF5 under data/fields/structured_grids.


In [1]:
from __future__ import annotations
from pathlib import Path
import os
os.chdir("../../..")
import sys
import yaml
import numpy as np
import h5py
from mpi4py import MPI

PROJECT_ROOT = Path.cwd().resolve()
sys.path.append(str(PROJECT_ROOT / "pySEMTools"))

from pysemtools.io.ppymech.neksuite import pynekread
from pysemtools.datatypes.msh import Mesh
from pysemtools.datatypes.field import FieldRegistry
from pysemtools.datatypes.utils import extrude_2d_sem_mesh
from pysemtools.interpolation.probes import Probes
import pysemtools.interpolation.pointclouds as pcs
import pysemtools.interpolation.utils as interp_utils

comm = MPI.COMM_WORLD
rank = comm.rank




### Load config and define paths


In [2]:
CONFIG_PATH = PROJECT_ROOT / "notebooks" / "preprocessing" / "nek_to_structured" / "nek_to_structured.yaml"
CFG = yaml.safe_load(CONFIG_PATH.read_text(encoding="utf-8"))

DATA_BASE_DIR = PROJECT_ROOT / Path(CFG["DATA_BASE_DIR"])
OUTPUT_BASE_DIR = PROJECT_ROOT / Path(CFG["OUTPUT_BASE_DIR"])

PHI = float(CFG["PHI"])
LAT_SIZE = str(CFG["LAT_SIZE"])
POST = bool(CFG["POST"])

FILE_PREFIX = CFG.get("FILE_PREFIX")
if not FILE_PREFIX:
    FILE_PREFIX = "po_postPremix0" if POST else "premix0"

FILE_INDEX_PAD = int(CFG.get("FILE_INDEX_PAD", 5))

GEOM_TIME_STEP = int(CFG.get("GEOM_TIME_STEP", 1))

EXTRUDE_2D = bool(CFG.get("EXTRUDE_2D", True))
EXTRUDE_LZ = CFG.get("EXTRUDE_LZ", None)
if EXTRUDE_LZ is not None:
    EXTRUDE_LZ = int(EXTRUDE_LZ)

MULTI_TIME_STEP = bool(CFG["MULTI_TIME_STEP"])
TIME_STEP = int(CFG["TIME_STEP"])
TIME_STEP_START = int(CFG["TIME_STEP_START"])
TIME_STEP_END = int(CFG["TIME_STEP_END"])

PHI_DIR = f"phi{PHI:.2f}"
LAT_TAG = f"h400x{LAT_SIZE}_ref"
CASE_DIR = DATA_BASE_DIR / PHI_DIR / LAT_TAG

mesh_file = CASE_DIR / f"{FILE_PREFIX}.f{str(GEOM_TIME_STEP).zfill(FILE_INDEX_PAD)}"

PHI_TAG = f"phi_{PHI:.2f}"
OUT_DIR = OUTPUT_BASE_DIR / PHI_TAG / LAT_TAG
OUT_DIR.mkdir(parents=True, exist_ok=True)

if rank == 0:
    if not mesh_file.exists():
        raise FileNotFoundError(f"Mesh/geometry file not found: {mesh_file}")
    print(f"Mesh file: {mesh_file}")

if rank == 0:
    print(f"Input case dir: {CASE_DIR}")
    print(f"Output dir: {OUT_DIR}")






Mesh file: /media/alexandros/OS/Users/alexp/Documents/Bachelor Thesis/Code/data/nek/phi0.40/h400x025_ref/premix0.f00001
Input case dir: /media/alexandros/OS/Users/alexp/Documents/Bachelor Thesis/Code/data/nek/phi0.40/h400x025_ref
Output dir: /media/alexandros/OS/Users/alexp/Documents/Bachelor Thesis/Code/data/fields/structured_grids/phi_0.40/h400x025_ref


### Build structured grid and write points file


In [3]:
X_BBOX = list(CFG["X_BBOX"])
Y_BBOX = list(CFG["Y_BBOX"])
Z_BBOX = list(CFG["Z_BBOX"])
NX = int(CFG["NX"])
NY = int(CFG["NY"])
NZ = int(CFG["NZ"])
GRID_MODE = str(CFG.get("GRID_MODE", "equal"))
GRID_GAIN = float(CFG.get("GRID_GAIN", 1.0))

x_1d = pcs.generate_1d_arrays(X_BBOX, NX, mode=GRID_MODE, gain=GRID_GAIN)
y_1d = pcs.generate_1d_arrays(Y_BBOX, NY, mode=GRID_MODE, gain=GRID_GAIN)
z_1d = pcs.generate_1d_arrays(Z_BBOX, NZ, mode=GRID_MODE, gain=GRID_GAIN)

x, y, z = np.meshgrid(x_1d, y_1d, z_1d, indexing="ij")

POINTS_FNAME = str(CFG.get("POINTS_FNAME", "points.hdf5"))
points_path = OUT_DIR / POINTS_FNAME

if rank == 0:
    with h5py.File(points_path, "w") as f:
        f.attrs["nx"] = NX
        f.attrs["ny"] = NY
        f.attrs["nz"] = NZ
        f.create_dataset("x", data=x)
        f.create_dataset("y", data=y)
        f.create_dataset("z", data=z)
    print(f"Wrote points file: {points_path}")

comm.Barrier()
if rank == 0:
    xyz = interp_utils.transform_from_array_to_list(NX, NY, NZ, [x, y, z])
else:
    xyz = None





Wrote points file: /media/alexandros/OS/Users/alexp/Documents/Bachelor Thesis/Code/data/fields/structured_grids/phi_0.40/h400x025_ref/points.hdf5


### Interpolate Nek file sequence onto structured grid


In [4]:
OUTPUT_FILE_STEM = str(CFG.get("OUTPUT_FILE_STEM", "structured_fields"))
output_file = OUT_DIR / f"{OUTPUT_FILE_STEM}.hdf5"

if MULTI_TIME_STEP:
    time_steps = list(range(TIME_STEP_START, TIME_STEP_END + 1))
else:
    time_steps = [TIME_STEP]

file_sequence = [
    CASE_DIR / f"{FILE_PREFIX}.f{str(ts).zfill(FILE_INDEX_PAD)}"
    for ts in time_steps
]

existing_files = [p for p in file_sequence if p.exists()]
if rank == 0:
    missing = [p for p in file_sequence if not p.exists()]
    if missing:
        print(f"Warning: {len(missing)} files missing; they will be skipped.")
        for p in missing[:5]:
            print(f"  missing: {p}")
    print(f"Interpolating {len(existing_files)} files.")

if not existing_files:
    raise FileNotFoundError("No Nek files found for interpolation.")

# Read mesh/geometry once (same procedure as extract_fields)
msh = Mesh(comm, create_connectivity=False)
pynekread(str(mesh_file), comm, msh=msh)

mesh_is_2d = (msh.x.shape[1] == 1)
if EXTRUDE_2D and mesh_is_2d:
    lz = EXTRUDE_LZ if EXTRUDE_LZ is not None else int(msh.lx)
    msh_use = extrude_2d_sem_mesh(comm, lz=lz, msh=msh)
    if rank == 0:
        print(f"Extruded 2D mesh to lz={lz} for interpolation.")
else:
    msh_use = msh
    lz = None
    if rank == 0 and mesh_is_2d:
        print("Mesh is 2D (lz=1). Set EXTRUDE_2D: true to avoid interpolation issues.")

# Initialize probes
probes = Probes(
    comm,
    output_fname=str(output_file),
    probes=xyz,
    msh=msh_use,
    write_coords=bool(CFG.get("WRITE_COORDS", True)),
    point_interpolator_type=str(CFG.get("POINT_INTERPOLATOR_TYPE", "multiple_point_legendre_numpy")),
    max_pts=int(CFG.get("MAX_PTS", 256)),
    find_points_iterative=list(CFG.get("FIND_POINTS_ITERATIVE", [False, 5000])),
    find_points_comm_pattern=str(CFG.get("FIND_POINTS_COMM_PATTERN", "point_to_point")),
    elem_percent_expansion=float(CFG.get("ELEM_PERCENT_EXPANSION", 0.01)),
    global_tree_type=str(CFG.get("GLOBAL_TREE_TYPE", "rank_bbox")),
    global_tree_nbins=int(CFG.get("GLOBAL_TREE_NBINS", 1024)),
    use_autograd=bool(CFG.get("USE_AUTOGRAD", False)),
    find_points_tol=float(CFG.get("FIND_POINTS_TOL", 1.0e-12)),
    find_points_max_iter=int(CFG.get("FIND_POINTS_MAX_ITER", 50)),
    local_data_structure=str(CFG.get("LOCAL_DATA_STRUCTURE", "kdtree")),
    use_oriented_bbox=bool(CFG.get("USE_ORIENTED_BBOX", False)),
)

fields_to_interpolate = CFG.get("FIELDS_TO_INTERPOLATE", ["all"])
if fields_to_interpolate != ["all"]:
    fields_to_interpolate = [str(f) for f in fields_to_interpolate]

for ts, fname in zip(time_steps, existing_files):
    # Use the timestep index for HDF5 filenames (match CSV convention).
    probes.written_file_counter = int(ts) - 1

    if rank == 0:
        print(f"=== Interpolating {fname.name} ===")

    fld = FieldRegistry(comm)
    pynekread(str(fname), comm, msh=msh, fld=fld)

    fld_use = fld
    if EXTRUDE_2D and mesh_is_2d:
        fld_use = extrude_2d_sem_mesh(comm, lz=lz, fld=fld)
        fld_use.t = fld.t

    if fields_to_interpolate == ["all"]:
        field_names = list(fld_use.registry.keys())
    else:
        field_names = []
        for name in fields_to_interpolate:
            if name not in fld_use.registry:
                raise KeyError(
                    f"Field {name!r} not found in registry. Available: {list(fld_use.registry.keys())}"
                )
            field_names.append(name)

    field_list = [fld_use.registry[name] for name in field_names]

    probes.interpolate_from_field_list(
        fld_use.t,
        field_list,
        comm,
        write_data=True,
        field_names=field_names,
    )






Interpolating 3 files.
2026-01-19 12:32:39,888 - Mesh - INFO - Initializing empty Mesh object.
2026-01-19 12:32:39,888 - pynekread - INFO - Reading file: /media/alexandros/OS/Users/alexp/Documents/Bachelor Thesis/Code/data/nek/phi0.40/h400x025_ref/premix0.f00001
2026-01-19 12:32:39,898 - Mesh - INFO - Initializing Mesh object from x,y,z ndarrays.
2026-01-19 12:32:39,899 - Mesh - INFO - Initializing common attributes.
2026-01-19 12:32:39,899 - Mesh - INFO - Getting vertices
2026-01-19 12:32:39,901 - Mesh - INFO - Getting edge centers
2026-01-19 12:32:39,925 - Mesh - INFO - Facet centers not available for 2D
2026-01-19 12:32:39,925 - Mesh - INFO - Mesh object initialized.
2026-01-19 12:32:39,925 - Mesh - INFO - Mesh data is of type: float64
2026-01-19 12:32:39,926 - Mesh - INFO - Elapsed time: 0.027527734999999998s
2026-01-19 12:32:39,926 - pynekread - INFO - File read
2026-01-19 12:32:39,926 - pynekread - INFO - Elapsed time: 0.038100831s
2026-01-19 12:32:39,954 - Mesh - INFO - Initiali

### Quick check (rank 0)


In [5]:
if rank == 0:
    first_ts = time_steps[0] if "time_steps" in globals() and time_steps else TIME_STEP
    first_out = OUT_DIR / f"{OUTPUT_FILE_STEM}{str(first_ts).zfill(5)}.hdf5"
    if first_out.exists():
        with h5py.File(first_out, "r") as f:
            keys = list(f.keys())
            print(f"First output: {first_out}")
            print(f"time={f.attrs.get('time', None)}")
            print(f"datasets={keys[:6]}")
    else:
        print("No output files found yet.")



First output: /media/alexandros/OS/Users/alexp/Documents/Bachelor Thesis/Code/data/fields/structured_grids/phi_0.40/h400x025_ref/structured_fields00197.hdf5
time=39.4000000001
datasets=['t']


In [6]:
if rank == 0:
    import matplotlib.pyplot as plt

    plot_file_index = CFG.get("PLOT_FILE_INDEX", 1)
    plot_file_index = int(plot_file_index) if plot_file_index is not None else 1
    plot_file = OUT_DIR / f"{OUTPUT_FILE_STEM}{str(plot_file_index).zfill(5)}.hdf5"
    plot_field = CFG.get("PLOT_FIELD", "")
    if plot_field is None:
        plot_field = ""
    plot_field = str(plot_field).strip()

    if not plot_file.exists():
        print(f"Plot file not found: {plot_file}")
    else:
        with h5py.File(plot_file, "r") as f:
            available_fields = [k for k in f.keys() if k not in ("x", "y", "z")]
            if not available_fields:
                raise ValueError(f"No field datasets found in {plot_file}")
            if not plot_field:
                plot_field = available_fields[0]
            if plot_field not in f:
                raise KeyError(
                    f"Requested PLOT_FIELD '{plot_field}' not found. Available: {available_fields}"
                )
            field = f[plot_field][:]
            plot_time = f.attrs.get("time", None)

        with h5py.File(points_path, "r") as f:
            x = f["x"][:]
            y = f["y"][:]
            z = f["z"][:]

        if field.ndim == 1:
            if field.size == x.size:
                field = field.reshape(x.shape)
            elif field.size == x.shape[0] * x.shape[1] and x.shape[2] == 1:
                field = field.reshape((x.shape[0], x.shape[1], 1))
            else:
                raise ValueError(
                    f"Field size {field.size} does not match grid size {x.size}."
                )
        elif field.ndim == 2:
            if x.shape[2] == 1 and field.shape == (x.shape[0], x.shape[1]):
                field = field[:, :, None]
            else:
                raise ValueError(
                    f"Field shape {field.shape} does not match grid shape {x.shape}."
                )
        elif field.ndim != 3:
            raise ValueError(
                f"Field has unsupported ndim={field.ndim}. Expected 1, 2, or 3."
            )

        z_idx_cfg = CFG.get("PLOT_SLICE_INDEX", None)
        z_idx = int(z_idx_cfg) if z_idx_cfg is not None else int(z.shape[2] // 2)
        z_idx = max(0, min(z.shape[2] - 1, z_idx))

        x_plot = x[:, :, z_idx]
        y_plot = y[:, :, z_idx]
        field_plot = field[:, :, z_idx]

        fig, ax = plt.subplots(figsize=(6, 3), dpi=150)
        pcm = ax.pcolormesh(x_plot, y_plot, field_plot, shading="auto", cmap="viridis")
        fig.colorbar(pcm, ax=ax, label=plot_field)
        ax.set_aspect("equal")
        time_label = f", t={plot_time:.6g}" if plot_time is not None else ""
        ax.set_title(f"{plot_field} (z index {z_idx}{time_label})")
        ax.set_xlabel("x")
        ax.set_ylabel("y")
        plt.tight_layout()
        plt.show()

    if ("probes" in globals()
            and hasattr(probes, "itp")
            and hasattr(probes.itp, "err_code")):
        err_code = probes.itp.err_code
        n_total = int(err_code.size)
        n_found = int(np.count_nonzero(err_code == 1))
        n_warning = int(np.count_nonzero(err_code == -10))
        n_missing = int(np.count_nonzero(err_code == 0))
        found_frac = n_found / n_total if n_total else 0.0
        warning_frac = n_warning / n_total if n_total else 0.0
        missing_frac = n_missing / n_total if n_total else 0.0
        print(
            "Interpolation coverage (point finding): "
            f"found={n_found}/{n_total} ({found_frac:.2%}), "
            f"warnings={n_warning} ({warning_frac:.2%}), "
            f"missing={n_missing} ({missing_frac:.2%})"
        )
    else:
        print("Interpolation coverage metric unavailable (probes not initialized).")


Plot file not found: /media/alexandros/OS/Users/alexp/Documents/Bachelor Thesis/Code/data/fields/structured_grids/phi_0.40/h400x025_ref/structured_fields00001.hdf5
Interpolation coverage (point finding): found=160000/160000 (100.00%), warnings=0 (0.00%), missing=0 (0.00%)
